In [1]:
import folium
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.plot import show
from rasterio.warp import transform_bounds
from matplotlib.colors import LinearSegmentedColormap


In [2]:
# Paths to the files
tiff_files = [
    r"D:\Acres\January 2025\Assignment 9\East Singbhum\EastSingbhum\EastSingbhum_B2.tif",
    r"D:\Acres\January 2025\Assignment 9\East Singbhum\EastSingbhum\EastSingbhum_B3.tif",
    r"D:\Acres\January 2025\Assignment 9\East Singbhum\EastSingbhum\EastSingbhum_B4.tif",
    r"D:\Acres\January 2025\Assignment 9\East Singbhum\EastSingbhum\EastSingbhum_B8.tif",
    r"D:\Acres\January 2025\Assignment 9\East Singbhum\EastSingbhum\EastSingbhum_B8_contrast.tif",
    r"D:\Acres\January 2025\Assignment 9\East Singbhum\EastSingbhum\EastSingbhum_B8_corr.tif",
    r"D:\Acres\January 2025\Assignment 9\East Singbhum\EastSingbhum\EastSingbhum_B8_diss.tif",
    r"D:\Acres\January 2025\Assignment 9\East Singbhum\EastSingbhum\EastSingbhum_B8_var.tif",
    r"D:\Acres\January 2025\Assignment 9\East Singbhum\EastSingbhum\EastSingbhum_NDVI.tif",
    r"D:\Acres\January 2025\Assignment 9\East Singbhum\EastSingbhum\EastSingbhum_NDWI.tif",
    r"D:\Acres\January 2025\Assignment 9\East Singbhum\EastSingbhum\predicted.tif",
]


In [3]:
shapefile_path = r"D:\Acres\January 2025\Assignment 9\East Singbhum\EastSingbhumVillage.shp"

# Load the shapefile
villages = gpd.read_file(shapefile_path)

# Get the list of village names
village_names = villages["NAME"].unique()


In [4]:
# Create a colormap (red to green)
cmap = LinearSegmentedColormap.from_list("RedGreen", ["red", "green"])


In [5]:
# Function to add TIFF layers to the map
def add_tiff_layer(map_obj, tiff_path, layer_name):
    with rasterio.open(tiff_path) as src:
        band = src.read(1)  # Read the first band
        band = np.where(band == src.nodata, np.nan, band)  # Handle nodata values
        bounds = transform_bounds(src.crs, "EPSG:4326", *src.bounds)

        folium.raster_layers.ImageOverlay(
            image=band,
            bounds=[[bounds[1], bounds[0]], [bounds[3], bounds[2]]],
            colormap=lambda x: cmap(x),
            name=layer_name,
            opacity=0.7
        ).add_to(map_obj)

In [6]:
# Function to create the map
def create_map():
    # Initialize the map
    center = [villages.geometry.centroid.y.mean(), villages.geometry.centroid.x.mean()]
    map_obj = folium.Map(location=center, zoom_start=12)

    # Add shapefile layers
    folium.GeoJson(
        villages,
        name="Villages",
        style_function=lambda feature: {
            'color': 'black', 'weight': 2, 'fillOpacity': 0
        },
        highlight_function=lambda feature: {
            'color': 'red', 'weight': 3
        },
        tooltip=folium.GeoJsonTooltip(fields=["NAME"], aliases=["Village Name"])
    ).add_to(map_obj)

    # Add TIFF layers
    for tiff in tiff_files:
        layer_name = tiff.split("\\")[-1]
        add_tiff_layer(map_obj, tiff, layer_name)

    # Add layer control
    folium.LayerControl().add_to(map_obj)

    return map_obj

# Function to zoom into a village
def zoom_to_village(map_obj, village_name):
    selected_village = villages[villages["NAME"] == village_name]
    if not selected_village.empty:
        bounds = selected_village.total_bounds
        map_obj.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])


In [ ]:
# Create the map
map_view = create_map()




In [ ]:
# Save the map to an HTML file and display it
map_view.save("dynamic_map.html")
print("Dynamic map saved as 'dynamic_map.html'. Open this file in a browser to view the map.")